In [100]:
import os
import requests
import re
import math
import easier as ezr
import itertools
import pandas as pd
import copy
from pprint import pprint
import time
import dataset
%config Completer.use_jedi = False

In [92]:
class GithubBase:
    BASE_URL = 'https://api.github.com/gists'
    USER_URL = 'https://api.github.com/users/robdmc/gists'
    REX_NEXT = re.compile(r'.*<(https.*?)>; rel="next"')
    PER_PAGE = 99
#     PER_PAGE = 9
    MAX_GISTS = 10_000
    
    LIST_TABLE = 'gist_list'
    GIST_TABLE = 'gist_table'
    
    pkc = ezr.pickle_cache_state('active')

    def __init__(self):
        self.access_token = os.environ['GIST_TOKEN']

    def blob_generator(self):
        for page in range(int(math.ceil(self.MAX_GISTS / self.PER_PAGE))):
            params = {
                'per_page': self.PER_PAGE,
                'page': page,
            }
            headers = {
                'Authorization': f'token {self.access_token}',
                'accept': 'application/vnd.github.v3+json'
            }
            resp = requests.get(self.USER_URL, params=params, headers=headers)
            self.latest_resp = resp
            blobs = resp.json()
            if blobs:
                yield blobs
            else:
                break
                
    def get_list(self):
        rec_list = []
        for ll in self.blob_generator():
            rec_list.extend(ll)
            
        return rec_list
    
    @ezr.pickle_cached_container()
    def ll(self):
        return self.get_list()
    
    def get_parsed_blob_list(self):
        keys = [
            'url',
            'id',
            'description',
            'created_at',
            'updated_at',

        ]
        records = []
        
        # for item in self.blob_generator():
        for item in self.ll[:5]:
            record_template = {k: item[k] for k in keys}
            for file in item['files'].values():
                record = record_template.copy()
                record['file'] = file['filename']
                record['language'] = file.get('language', 'unkown')
                record['file_url'] = file['raw_url']
                record['size'] = file.get('size', 0)
                record['gist_id'] = record.pop('id')
                record['code'] = ''
                records.append(record)
                
        dedup = {r['file_url']: r for r in records}
        records = list(dedup.values())
                
                
        return records



gh = GithubBase()
recs = gh.get_parsed_blob_list()
print(recs[0])

{'url': 'https://api.github.com/gists/5a795e497e3ea6e9bf408a1635694067', 'description': 'Holoviews stacked area plot', 'created_at': '2021-01-21T00:57:58Z', 'updated_at': '2021-01-30T19:34:17Z', 'file': 'holoviews_stacked_area_chart.py', 'language': 'Python', 'file_url': 'https://gist.githubusercontent.com/robdmc/5a795e497e3ea6e9bf408a1635694067/raw/601076806b5f19f82beb6d0d0595d40666ee1b8d/holoviews_stacked_area_chart.py', 'size': 210, 'gist_id': '5a795e497e3ea6e9bf408a1635694067', 'code': ''}


In [117]:
db = dataset.connect('sqlite:///:memory:')
list_table_name = 'list_table'
code_table_name = 'code_table'

def populate_list(recs, print=False):
    table = db[list_table_name]
    table.drop()
    table.insert_many(recs)
    

# def populate_code(recs):
#     list_table = db[list_table_name]
#     code_table = db[code_table_name]
#     input_file_urls = {r['file_url'] for r in recs}
#     existing_file_url_lookup = {r['file_url']: r for r in list_table.all()}
#     existing_file_urls = set(existing_file_url_lookup.keys())
    
#     new_file_urls = input_file_urls - existing_file_urls
#     dead_file_urls = existing_file_urls - input_file_urls
    
#     dead_file_url_ids = [existing_file_url_lookup[url]['id'] for url in dead_file_urls]
    
#     for dead_id in dead_file_url_ids:
#         code_table.delete(id=dead_id)
    
    
# #     print(new_file_urls)
#     print(new_file_urls, dead_file_urls, dead_file_url_ids)


def get_lookup_dict(table):
    return {r['file_url']: r for r in table.all()}

def get_code(url):
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.text
    else:
        return None
    
def update_code_table(verbose=True):
    list_table = db[list_table_name]
    code_table = db[code_table_name]
    
    code_lookup = get_lookup_dict(code_table)
    list_lookup = get_lookup_dict(list_table)
    
    missing_file_urls = set(list_lookup.keys()) - set(code_lookup.keys())
    deleted_file_urls = set(code_lookup.keys()) - set(list_lookup.keys())
    
    code_table.delete(file_url={'in': tuple(deleted_file_urls)})
    
    if not missing_file_urls:
        return
    
    if verbose:
        missing_file_urls = tqdm(missing_file_urls)
    
    for file_url in missing_file_urls:
        code = get_code(file_url)
        code_table.insert(dict(file_url=file_url, code=code))
        time.sleep(.1)
        
    code_table.create_index('file_url')
    
    
    
populate_list(recs)

new_recs = copy.deepcopy(recs[:-1])
new_recs[0]['file_url'] = 'lucy'
update_code_table()
update_code_table()


100%|██████████| 6/6 [00:01<00:00,  4.45it/s]


In [114]:
print(db['code_table'].find_one()['code'])

c_list = []
for col in df.columns[1:]:
    c = hv.Area((df.date, df[col]/1e6), 'Date', 'Active Pipeline ($M)', label=col).opts(color=hv.Cycle('Set1'))
    c_list.append(c)
    
hv.Area.stack(hv.Overlay(c_list))


In [103]:
list(db['code_table'].distinct('file_url'))

[OrderedDict([('file_url',
               'https://gist.githubusercontent.com/robdmc/32275939f121fc7f5661967cc321527f/raw/ea33d1ad4792a6af74e1d100071195cebca4cf4c/altair_plotting_example.py')]),
 OrderedDict([('file_url',
               'https://gist.githubusercontent.com/robdmc/3f82bc492e87a1d5955e43250b32051c/raw/01f3ef69e4d9c3769ea2cd45951dabb69650a284/networkx_plot_graphviz.py')]),
 OrderedDict([('file_url',
               'https://gist.githubusercontent.com/robdmc/45cc1644d15c70aedf7b432d7ade825d/raw/cefbc40549b6fb2b23a1baf962a16f73ec41e66b/holoviews_color_cycle_color_names.py')]),
 OrderedDict([('file_url',
               'https://gist.githubusercontent.com/robdmc/5a795e497e3ea6e9bf408a1635694067/raw/601076806b5f19f82beb6d0d0595d40666ee1b8d/holoviews_stacked_area_chart.py')]),
 OrderedDict([('file_url',
               'https://gist.githubusercontent.com/robdmc/5a795e497e3ea6e9bf408a1635694067/raw/81f87b8d96081415c4fc1bec9bc27cd063ff24cc/readme.md')]),
 OrderedDict([('file_url',
 

In [95]:
df[df.file_url == 'https://gist.githubusercontent.com/robdmc/5b47c50811f0118bfd00cdfaef44e141/raw/8340a1d66084e1753883c98ba09e754ef4393127/ambition_debug_doit_script.py']

,url,description,created_at,updated_at,file,language,file_url,size,gist_id,code
13,https://api.github.com/gists/5b47c50811f0118bf...,Script to debug ambition,2019-10-08T15:22:00Z,2019-10-08T15:22:00Z,ambition_debug_doit_script.py,Python,https://gist.githubusercontent.com/robdmc/5b47...,281,5b47c50811f0118bfd00cdfaef44e141,
105,https://api.github.com/gists/5b47c50811f0118bf...,Script to debug ambition,2019-10-08T15:22:00Z,2019-10-08T15:22:00Z,ambition_debug_doit_script.py,Python,https://gist.githubusercontent.com/robdmc/5b47...,281,5b47c50811f0118bfd00cdfaef44e141,


In [96]:

db = dataset.connect('sqlite:///:memory:')
table_name = 'rob'
table = db[table_name]
table.drop()
table.insert_many(recs)
table.find_one()

OrderedDict([('id', 1),
             ('url',
              'https://api.github.com/gists/5a795e497e3ea6e9bf408a1635694067'),
             ('description', 'Holoviews stacked area plot'),
             ('created_at', '2021-01-21T00:57:58Z'),
             ('updated_at', '2021-01-30T19:34:17Z'),
             ('file', 'holoviews_stacked_area_chart.py'),
             ('language', 'Python'),
             ('file_url',
              'https://gist.githubusercontent.com/robdmc/5a795e497e3ea6e9bf408a1635694067/raw/601076806b5f19f82beb6d0d0595d40666ee1b8d/holoviews_stacked_area_chart.py'),
             ('size', 210),
             ('gist_id', '5a795e497e3ea6e9bf408a1635694067'),
             ('code', '')])

In [42]:
existing_file_urls = {r['file_url'] for r in table.all()}
new_file_urls = {r['file_url'] for r in new_recs}

In [43]:
to_upsert = new_file_urls - existing_file_urls
to_delete = existing_file_urls - new_file_urls
to_upsert, to_delete

({'lucy'}, set())

In [38]:
len(existing_file_urls), len(new_file_urls)

(92, 93)

In [20]:
table_name = 'rob'
table = db[table_name]

def get_key(r):
    return (r['gist_id'], r['file'])

existing_rec_lookup = {get_key(r): r for r in table.all()}


In [21]:
to_upsert = []

class Sentinal:
    pass

for new_rec in new_recs:
    if existing_rec_lookup.get(get_key(new_rec), {'file_url': Sentinal()})['file_url'] != new_rec['file_url']:
        to_upsert.append(new_rec)
    
    

In [22]:
to_upsert

[{'url': 'https://api.github.com/gists/5a795e497e3ea6e9bf408a1635694067',
  'description': 'Holoviews stacked area plot',
  'created_at': '2021-01-21T00:57:58Z',
  'updated_at': '2021-01-21T00:57:59Z',
  'file': 'holoviews_stacked_area_chart.py',
  'language': 'Python',
  'file_url': 'lucy',
  'size': 210,
  'gist_id': '5a795e497e3ea6e9bf408a1635694067'}]

In [ ]:
existing_recs.p

In [50]:
rec

{'url': 'https://api.github.com/gists/0b97398a4e6950ce6928',
 'id': '0b97398a4e6950ce6928',
 'description': 'values_list_queryset_chunker',
 'created_at': '2015-03-23T20:16:55Z',
 'updated_at': '2015-08-29T14:17:39Z',
 'file': 'values_list_queryset_chunker.py',
 'language': 'Python',
 'file_url': 'https://gist.githubusercontent.com/robdmc/0b97398a4e6950ce6928/raw/72420c59954202d22a2976d8b873cb08c2dd6a8f/values_list_queryset_chunker.py',
 'size': '518'}

In [18]:
from pony import orm

In [20]:
db = orm.Database()
db.bind(provider='sqlite', filename=':memory:')

class Entry(db.Entity):
    gist_url = orm.Required(str)
    gist_id = orm.Required(str)
    gist_description = orm.Required(str)
    
    pass

In [10]:
import json
with open('/tmp/ghblob.json', 'w') as buff:
    json.dump(gh.ll, buff, indent=2)



In [13]:
xxx = requests.get('https://gist.githubusercontent.com/robdmc/5a795e497e3ea6e9bf408a1635694067/raw/601076806b5f19f82beb6d0d0595d40666ee1b8d/holoviews_stacked_area_chart.py')

In [17]:
print(xxx.text)

c_list = []
for col in df.columns[1:]:
    c = hv.Area((df.date, df[col]/1e6), 'Date', 'Active Pipeline ($M)', label=col).opts(color=hv.Cycle('Set1'))
    c_list.append(c)
    
hv.Area.stack(hv.Overlay(c_list))


In [11]:
type(resp.headers)

requests.structures.CaseInsensitiveDict

In [37]:
blob = resp.json()

keys = [
    'url',
    'description',
    'created_at',
    'updated_at',
    
]
records = []
for item in blob:
    record_template = {k: item[k] for k in keys}
    for file in item['files'].values():
        record = record_template.copy()
#         print(file)
        record['file'] = file['filename']
        record['language'] = file.get('language', 'unkown')
        record['file_url'] = file['raw_url']
        record['size'] = file.get('size', 0)
        records.append(record)
#     pprint(item)

rec = records[0]
rec


{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
 'description': 'Visidata Monstrosity',
 'created_at': '2021-01-17T17:14:45Z',
 'updated_at': '2021-01-17T19:55:10Z',
 'file': 'monstrous_visidatarc.py',
 'language': 'Python',
 'file_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
 'size': 1999}

In [16]:
blob[0]

{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
 'forks_url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84/forks',
 'commits_url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84/commits',
 'id': '611b57470ca1522c4f3c496a7c967a84',
 'node_id': 'MDQ6R2lzdDYxMWI1NzQ3MGNhMTUyMmM0ZjNjNDk2YTdjOTY3YTg0',
 'git_pull_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84.git',
 'git_push_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84.git',
 'html_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84',
 'files': {'monstrous_visidatarc.py': {'filename': 'monstrous_visidatarc.py',
   'type': 'application/x-python',
   'language': 'Python',
   'raw_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
   'size': 1999}},
 'public': True,
 'created_at': '2021-01-17T17:14:45Z',
 'updated_at': '2021-01-17T19:55:

In [2]:
from visidata import SequenceSheet, BaseSheet, Path, Column, asyncthread, TableSheet
from visidata import vd


In [3]:

# def rowtype(*args, **kwargs):
class RobSheet(TableSheet):
    _rowtype = object  # rowdef: list of values

    

    def iterload(self):
        for nn in range(3):
            yield  [nn, nn, nn]

    #     @asyncthread
    def reload(self):
        self.columns = []
        for col in ['a', 'b', 'c']:
            self.addColumn(Column(col))
        
        self.rows = []
        for row in self.iterload():
            self.addRow(row)
            
        

In [4]:
rs = RobSheet()
t = rs.reload()

# for col in rs.it:
#     print(repr(col))


In [5]:
for col in rs.columns:
    print(col.name)

a
b
c


In [6]:
for typedvals in rs.iterdispvals(format=False):

    for col, val in typedvals.items():
        print(col.name, val, type(val))

a [0, 0, 0] <class 'list'>
b [0, 0, 0] <class 'list'>
c [0, 0, 0] <class 'list'>
a [1, 1, 1] <class 'list'>
b [1, 1, 1] <class 'list'>
c [1, 1, 1] <class 'list'>
a [2, 2, 2] <class 'list'>
b [2, 2, 2] <class 'list'>
c [2, 2, 2] <class 'list'>


In [154]:
vd.save_csv(Path('/tmp/jerry.csv'), rs)

In [161]:
nr = rs.newRow()

In [162]:
nr[['a', 'b']]

TypeError: unhashable type: 'list'

In [160]:
nr

BasicRow(<function visidata.sheets.BasicRow.__init__.<locals>.<lambda>()>,
         {'rob': None})